In [4]:
url = "https://www.govtrack.us/congress/members/all#all_role_types=2&all_role_states=NY"
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import time
from rough_functions import * 
from new_functions import * 


In [55]:
main_data = readData()
main_data=main_data.drop(main_data[main_data['ticker'].isna()].index, axis=0)
unique_names = np.unique(main_data['name'])
print("# of Unique Names: "+str(len(unique_names)))
print(unique_names)


# of Unique Names: 228
['Abigail Spanberger' 'Adam B. Schiff' 'Adam Kinzinger' 'Adrian Smith'
 'Alan S. Lowenthal' 'Andrew Garbarino' 'Angus S. King, Jr.' 'Ann Wagner'
 'Anthony E. Gonzalez' 'Ashley Hinson Arenholz' 'Aston Donald McEachin'
 'August Lee Pfluger' 'Austin Scott' 'Benjamin L Cardin' 'Bill Cassidy'
 'Bill Flores' 'Bill Hagerty' 'Bill Huizenga' 'Bill Pascrell' 'Billy Long'
 'Blake Moore' 'Bob Gibbs' 'Brad Sherman' 'Bradley S. Schneider'
 'Brian Higgins' 'Brian Mast' 'Carol Devine Miller' 'Carolyn B. Maloney'
 'Charles J. "Chuck" Fleischmann' 'Chellie Pingree' 'Cheri Bustos'
 'Chris Jacobs' 'Chris Van Hollen' 'Christopher A. Coons' 'Cliff Bentz'
 'Cory A Booker' 'Cynthia Axne' 'Cynthia M. Lummis' 'Dan Crenshaw'
 'Dan Daniel Bishop' 'Dan Newhouse' 'Dan Sullivan' 'Daniel Meuser'
 'David B. McKinley' 'David Cheston Rouzer' 'David E. Price'
 'David Kustoff' 'David Madison Cawthorn' 'David P. Joyce' 'David P. Roe'
 'David Perdue' 'David Trone' 'Dean Phillips' 'Deb Fischer'
 'Debbi

In [31]:
data = {}


In [58]:
def name_to_working(url):# either returns text of ul object or returns a NoneType
    response=requests.get(url=url)
    soup2 = BeautifulSoup(response.text, 'html.parser')
    
    header = soup2.find("span",attrs={"id":"Committee_assignments"})
    if header:
        return header.findNext('ul').text.lower()
    
    header = soup2.find("span",attrs={"id":"Committee_assignment"})
    if header:
        return header.findNext('ul').text.lower()
    
    header = soup2.find("span",attrs={"id":"Current_committee_assignments"})
    if header:
        return header.findNext('ul').text.lower()
    
    header = soup2.find("span",attrs={"id":"Committees"})
    if header:
        return header.findNext('ul').text.lower()

    header = soup2.find("span", text="Committee assignments")
    if header:
        return header.findNext('ul').text.lower()
    
    header = soup2.find("span", text="Committee assignments")
    if header:
        return header.findNext('ul').text.lower()

    header = soup2.find("b", text="Committee assignments")
    if header:
        return header.findNext('ul').text.lower()

    return "Cannot Find Data"



def raw_to_committee_data(raw):
    committee_data = {}
    for i in raw.split('\n'):
        if i[0:5]=='commi' or i[0:5]=='unite':
            committee_data[i.split('committee on the ')[-1].split('committee on ')[-1]]=[]
        elif i[0:5]=='subco':
            committee_data.get(list(committee_data.keys())[-1]).append(i.replace('subcommittee on ',''))
    return committee_data

In [61]:
for i in unique_names:
    search = 'Wikipedia politician '+str(list(main_data.loc[main_data['name']==unique_names[0]]['state'])[0])+" "+i.replace('"','')
    url = 'https://www.google.com/search'

    parameters = {'q': search}

    content = requests.get(url, params = parameters).text
    soup = BeautifulSoup(content, 'html.parser')
    links = soup.find_all('a')

    for link in links:

        if "https://en.wikipedia.org/wiki/" in link['href']:
            
            link=(link['href'].split('%')[0].split('&')[0].split('q=')[1])

            raw = name_to_working(link)
            com_data=raw_to_committee_data(raw)

            d={
                'wiki_link':link,
                'raw':raw,
                'committee_data':com_data
                }
            print(d)
            data[i]=d
            break


{'wiki_link': 'https://en.wikipedia.org/wiki/Abigail_Spanberger', 'committee_data': {'agriculture': ['commodity exchanges, energy, and credit', 'conservation and forestry (chair)'], 'foreign affairs': ['asia, the pacific and nonproliferation', 'europe, eurasia, energy, and the environment']}}
{'wiki_link': 'https://en.wikipedia.org/wiki/Adam_Schiff', 'committee_data': {}}
{'wiki_link': 'https://en.wikipedia.org/wiki/Adam_Kinzinger', 'committee_data': {'energy and commerce[74]': ['communications and technology', 'energy'], 'foreign affairs[74]': ['the middle east, north africa, and international terrorism', 'europe, eurasia, energy, and the environment (ranking member)'], 'january 6 attack': []}}
{'wiki_link': 'https://en.wikipedia.org/wiki/Adrian_Smith_(politician)', 'committee_data': {'ways and means': ['trade', 'social security', 'health']}}
{'wiki_link': 'https://en.wikipedia.org/wiki/Alan_Lowenthal', 'committee_data': {}}
{'wiki_link': 'https://en.wikipedia.org/wiki/Andrew_Garbarin

IndexError: list index out of range

In [53]:
data

{}

Committee on the Judiciary
Subcommittee on Crime, Terrorism and Homeland Security
Subcommittee on Antitrust, Commercial and Administrative Law
Subcommittee on Immigration and Citizenship
Committee on Oversight and Reform
Subcommittee on Government Operations
Committee on Veterans' Affairs
Subcommittee on Disability Assistance and Memorial Affairs
Subcommittee on Health


In [33]:
for d in data.keys():
    if data.get(d).get('rough')==None:
        print(d)

In [111]:
def scraper(full_name):
    base_wiki = "https://en.wikipedia.org/wiki/"
    name=full_name.split(', ')[0]
    # print(str(i)+'  '+name)
    url=base_wiki+name.split(' ')[0]+"_"+name.split(' ')[-1]
    basic = name_to_working(url) #no manipulations to name/url
    if basic:
        return basic
    
    #try disambiguation attachment
    url_dis=url+'_(politician)'
    disambiguation = name_to_working(url_dis)
    if disambiguation:
        return disambiguation
    
    #try second/middle name
    url=base_wiki+name.split(' ')[1]+"_"+name.split(' ')[-1]
    second_name=name_to_working(url)
    if second_name:
        return second_name

    #try middle name
    if(len(name.split(' '))>2):
        url=base_wiki+name.split(' ')[0]+"_"+name.split('.')[0][-1]+"._"+name.split(' ')[-1]
        middle_name=name_to_working(url)
        if middle_name:
            return middle_name
    #try three names
        url=base_wiki+name.split(' ')[0]+"_"+name.split(' ')[-2]+"_"+name.split(' ')[-1]
        three_names=name_to_working(url)
        if three_names:
            return three_names
    #try first two names
    url=base_wiki+name.split(' ')[0]+"_"+name.split(' ')[1]
    first_two=name_to_working(url)
    if first_two:
        return first_two
    


    name_changes ={
        'Robert P. Casey, Jr.':'Bob_Casey_Jr.',
        "Joseph P. Kennedy III":"Joe_Kennedy_III"
    }
##SPECIAL ONE OFFS
    if(full_name in list(name_changes.keys())):
        url=base_wiki+name_changes.get(full_name)
        out=name_to_working(url)
        if out:
            return out

    return basic #give up



In [112]:
failed_names = []
for i in range(len(unique_names)):
    if not (scraper(unique_names[i])):
        print(unique_names[i])
        failed_names.append(unique_names[i])


Ashley Hinson Arenholz
Cynthia M. Lummis
Dan Newhouse
Dan Sullivan
David Kustoff
David Perdue
Garret Graves
Greg Francis Murphy
James M. Inhofe
John Curtis
John Kennedy
John Rutherford
Joseph P. Kennedy III
Michael B. Enzi
Mike Kelly
Mikie Sherrill
Richard W. Allen
Rudy Yakym
Thom Tillis
Tom Cole
Vern Buchanan


In [114]:
print (len(failed_names)/len(unique_names))

0.09210526315789473


In [ ]:

options = webdriver.ChromeOptions() 
# options.add_argument('--user-data-dir=C:/Users/Rae-A/AppData/Local/Google/Chrome/User Data')
# options.add_argument('--profile-directory=Default')
# options.add_argument("--headless")
#options.add_experimental_option("useAutomationExtension", False)
#options.add_experimental_option("excludeSwitches",["enable-automation"])
driver = webdriver.Chrome(options=options)
driver.get(url)
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
text = driver.find_element(By.XPATH, "/html/body/div[1]/div[4]/div[2]/section/div/div[2]").text
num_from_state = (len(text.split('\n'))/2) ##this gives the number of representatives from the state found


In [ ]:
for i in range(0,int(num_from_state),2):
    year = int(text.split('\n')[i+1].split(', ')[-1][-4:])
    name = text.split('\n')[i]
    if year>=2013:
        print(name+" - "+str(year))
